# ** RETO (opcional): Implementación algoritmo de descenso de gradiente en problema con múltiples minínimos locales **
**Facultad de ingeniería, departamento de Ingeniería Biomédica, Universidad de los Andes**\
**IBIO-2440 Programación científica**

**Nombres de los integrantes**


1.   David Tobón Molina
2.   David Santiago Rodríguez Quiroga

**Número del grupo**

Grupo 2

Considere el problema de optimización:



> $\min_x f(x)$ sujeto a $x\in\mathbb{R}^3$

donde $f:\mathbb{R^3}→\mathbb{R}$ y $x=[x_1,x_2,x_3]^T$. Note que este es un problema de minmización sin restricciones en $\mathbb{R}^3$. Se sabe que la función es derivable y posee varios minimizadores locales entre los cuales hayun minimizador global, todos con posiciones desconocidas. El objetivo del presente laboratorio bono es implementar y utilizar el algoritmo de descenso de gradiente para identificar la mayor cantidad de candidatos a minimizadores locales dentro de una región dada e identificar el posible minimizador global. 

*   Implemente su propia rutina del algoritmo de descenso de gradiente teniendo en cuenta que los minimizadores de la función están definida por $x_i ∈ [-5, 5]$ para cada $i = 1, …, 3$. Es decir, cada componente de $x$ està entre $-5$ y $5$. Defina un tamaño del salto $\alpha$ y un parámetro de criterio de parada $ϵ$ que usted considere conveniente para resolver el problema. Estime el gradiente de la función evaluado en un punto de forma numérica (como se hizo en una práctica anterior), y bajo ninguna circunstancia utilice variables simbólicas.
   
*    Corra el algoritmo de descenso de gradiente al menos para 500 condiciones iniciales diferentes, y guarde las soluciones resultantes de cada corrida (ya sea definidas de forma aleatoria o haciendo un recorrido definido en la región donde se sabe están los minimizadores locales). Basados en estos resultados, realice una tabla donde incluya los valores aproximados de los candidatos a minimizadores locales (recuerden que cada candidato es un vector 3-dimensional), el candidato a minimizador global, y el respectivo número de iteraciones que el algoritmo de descenso de gradiente requirió para llegar a la respuesta.

\\

Esta actividad funciona como un bono para la nota del primer parcial de los integrantes del grupo. La nota se asignará dependiendo de el correcto seguimiento de las instrucciones anteriores y la cantidad de minimizadores locales reales que su algoritmo encuentre. Cada grupo debe desarrollar su propia solución y no se permitirá copia entre grupos de todo el curso. Una falta ante esto se reportará inmediatamente ante el comité de ética de la facultad. 



In [138]:
import benchmark_functions as bf
import numpy as np

# func es el objeto creado con la funcion a minimizar
func = bf.StyblinskiTang(n_dimensions=3)
print(func.minimum())

(-117.4984971113142, [-2.903534, -2.903534, -2.903534])


In [139]:
# Dado un punto x0, func permite evaluar la función objetivo en x0 
x0 = np.array([5.0, 5.0, 5.0]) 
f=func(x0)
print(f)

375.0


In [140]:
def numeric_gradient(xi, f, e=0.0001):

    gradient_result = np.zeros(len(xi))
    
    for i in range(len(xi)):
        xi_k = np.copy(xi)
        xi_k[i] = xi_k[i]+e
        gradient_result[i] = (f(xi_k)-f(xi))/e

    return gradient_result


numeric_gradient(x0, func)

array([172.5067001, 172.5067001, 172.5067001])

In [143]:
def gradient_descent(f, numeric_gradient, xi, max_iter=500):

    alpha = None
    epsilon = None
    x_min = np.copy(xi)
    x = np.copy(xi)
    f_min = float(f(xi))
    f_eval = float(f(xi))
    
    for i in range(1, 10):
        alpha = i/10
        for j in range(1,10):

            epsilon = j/100
            n = 0
            stop = False

            while n < max_iter and (not stop):

                num_grad = numeric_gradient(x, f)
                xk_1= x - alpha*num_grad
                print(xk_1, x)
                resta = xk_1-x
                norma_l2 = np.sqrt(abs(resta[0])**2 + abs(resta[1])**2 + abs(resta[2])**2)
                
                if norma_l2 <= epsilon:
                    stop = True
                    f_eval = float(f(xk_1))
                    print(f_eval)
                    
                x = np.copy(xk_1)
                n +=1
                
            if f_eval < f_min:
                f_min = f_eval
                x_min = np.copy(x)
                
    return x_min, f_min
            
gradient_descent(func, numeric_gradient, x0)

[-12.25067001 -12.25067001 -12.25067001] [5. 5. 5.]
[335.60728994 335.60728994 335.60728995] [-12.25067001 -12.25067001 -12.25067001]
[-7559172.09076914 -7559172.09076914 -7559172.09076913] [335.60728994 335.60728994 335.60728995]
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [-7559172.09076914 -7559172.09076914 -7559172.09076913]
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.63875291e+19]
8.354019175475834e+79
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.63875291e+19]
8.354019175475834e+79
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.63875291e+19]
8.354019175475834e+79
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.63875291e+19]
8.354019175475834e+79
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.63875291e+19]
8.354019175475834e+79
[8.63875291e+19 8.63875291e+19 8.63875291e+19] [8.63875291e+19 8.63875291e+19 8.638752

(array([5., 5., 5.]), 375.0)